# Extreme Water level 100yr - RP

Notebook environment to migrate csv files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Import standard packages
import os
import pathlib
import sys

import numpy as np
#import geopandas as gpd
 
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import itertools
import json
import copy
from itertools import chain
from shapely import wkb

# Import custom functionality
# sys.path.append('c:/Windows/System32/coclicodata/src')#CLENMAR ADDED

from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = pathlib.Path.home().joinpath("Documents","GitHub")
#gca_data_dir = p_drive.joinpath("11209197-018-global-coastal-atlas","MSc_students","ClenmarRowe")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath(r"All_Datasets")
dataset_dir_path = dataset_dir.joinpath("NetCDFs","Hazards","EWL_100","EWL_100_original.nc")
CF_dir = dataset_dir.joinpath("NetCDFs","Hazards","EWL_100","CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [4]:
# # Project paths & files (manual input)
# dataset_dir = gca_data_dir.joinpath(r"All_Datasets")
# dataset_dir_path = dataset_dir.joinpath("EWL_100_original.nc")
# CF_dir = gca_data_dir.joinpath("Example_Coastalrisk_STAC","CFclen")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [5]:
filename=dataset_dir.joinpath("GCC datasets","GCC_hydrometeorological.csv")
template_filename=dataset_dir.joinpath("GCC datasets","Standard_1km_template.csv")

df = pd.read_csv(filename, index_col=None, header=0)
columns_to_remove = range(4, len(df.columns)-1)
df = df.drop(df.columns[columns_to_remove], axis=1)
df

,id,lon,lat,angle,wl_rp100_mean
0,BOX_S12E050_0001,51.167463,-10.163451,298.667671,NaN
1,BOX_S12E050_0002,51.164636,-10.172016,293.360252,NaN
2,BOX_S12E050_0003,51.160440,-10.180046,298.904589,NaN
3,BOX_S12E050_0004,51.156007,-10.187953,298.609831,NaN
4,BOX_S12E050_0005,51.152063,-10.196096,288.514959,NaN
...,...,...,...,...,...
728083,BOX_N40E038_0182,38.819165,41.006168,358.226533,0.199
728084,BOX_N40E038_0183,38.807751,41.003857,336.282428,0.199
728085,BOX_N40E038_0184,38.797307,40.999566,328.135790,0.199
728086,BOX_N40E038_0185,38.787499,40.994478,323.757427,0.199


<IPython.core.display.Javascript object>

In [6]:
df_t = pd.read_csv(template_filename, index_col=None, header=0)
df_t=df_t.drop(df_t.columns[0], axis=1)


<IPython.core.display.Javascript object>

In [7]:
df_t=df_t.assign(EWL_100=df['wl_rp100_mean'])


<IPython.core.display.Javascript object>

In [8]:
ds = xr.Dataset.from_dataframe(df_t)
ds

<xarray.Dataset>
Dimensions:  (index: 728088)
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 ... 728083 728084 728085 728086 728087
Data variables:
    id       (index) object 'BOX_S12E050_0001' ... 'BOX_N40E038_0186'
    lon      (index) float64 51.17 51.16 51.16 51.16 ... 38.81 38.8 38.79 38.78
    lat      (index) float64 -10.16 -10.17 -10.18 -10.19 ... 41.0 40.99 40.99
    angle    (index) float64 298.7 293.4 298.9 298.6 ... 336.3 328.1 323.8 320.9
    country  (index) object 'Seychelles' 'Seychelles' ... 'Türkiye' 'Türkiye'
    EWL_100  (index) float64 nan nan nan nan nan ... 0.199 0.199 0.199 0.198

<IPython.core.display.Javascript object>

In [9]:
# Write the xarray dataset to a netCDF file
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

ds.to_netcdf(str(dataset_dir_path))

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [10]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:  (index: 728088)
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 ... 728083 728084 728085 728086 728087
Data variables:
    id       (index) <U16 ...
    lon      (index) float64 ...
    lat      (index) float64 ...
    angle    (index) float64 ...
    country  (index) <U52 ...
    EWL_100  (index) float64 ...

<IPython.core.display.Javascript object>

In [11]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [12]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path, working_dir=CF_dir)

{'C:\\Users\\rowe\\Documents\\GitHub\\All_Datasets\\NetCDFs\\Hazards\\EWL_100\\EWL_100_original.nc': {'warnings': '11', 'errors': '2'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [13]:
import json

# NetCDF attribute alterations by means of metadata template
metadata_json=dataset_dir.joinpath("GCC datasets","Metadata","01_EWL_100_metadata.json")

f_global = open(metadata_json)
meta_global = json.load(f_global)


for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds.attrs[attr_name] = attr_val

ds.attrs['Conventions'] = "CF-1.8"
ds

<xarray.Dataset>
Dimensions:  (index: 728088)
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 ... 728083 728084 728085 728086 728087
Data variables:
    id       (index) <U16 ...
    lon      (index) float64 ...
    lat      (index) float64 ...
    angle    (index) float64 ...
    country  (index) <U52 ...
    EWL_100  (index) float64 ...
Attributes: (12/21)
    TITLE:               Extreme water level_ 100 RP
    TITLE_ABBREVIATION:  EWL
    DESCRIPTION:         Critical Risk Index (CRI) indicates risk by combinin...
    SHORT_DESCRIPTION:   Critical Risk Index (CRI) indicates risk by combinin...
    INSTITUTION:         IHE, Deltares
    PROVIDERS:           {"name": "clenclen", "url": "www.ivm.nl", "roles": "...
    ...                  ...
    DOI:                 https://doi.org/10.5281/zenodo.4957646
    LONG_NAME:           CRITICAL_INFRASTRUCTURE_SPATIAL_INDEX
    UNITS:               
    COMMENT:             The CISI is expressed in a dimensionless value rangi...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [14]:
ds = ds.rename_dims({"index": "nstations"})

# rename variables, if necessary
ds = ds.rename_vars(
      {"country":"stations"}, # {"X": "lon", "Y": "lat"} #"index":"nstations", 
  )
ds=ds.drop(["angle"])
ds=ds.drop(["index"])
ds=ds.drop(["id"])
# set some data variables to coordinates to avoid duplication of dimensions in later stage
ds = ds.set_coords(["lon", "lat","stations"])

ds

C:\Users\rowe\AppData\Local\Temp\ipykernel_9768\4214212903.py:7: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds=ds.drop(["angle"])
C:\Users\rowe\AppData\Local\Temp\ipykernel_9768\4214212903.py:8: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds=ds.drop(["index"])
C:\Users\rowe\AppData\Local\Temp\ipykernel_9768\4214212903.py:9: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds=ds.drop(["id"])


<xarray.Dataset>
Dimensions:   (nstations: 728088)
Coordinates:
    lon       (nstations) float64 ...
    lat       (nstations) float64 ...
    stations  (nstations) <U52 ...
Dimensions without coordinates: nstations
Data variables:
    EWL_100   (nstations) float64 ...
Attributes: (12/21)
    TITLE:               Extreme water level_ 100 RP
    TITLE_ABBREVIATION:  EWL
    DESCRIPTION:         Critical Risk Index (CRI) indicates risk by combinin...
    SHORT_DESCRIPTION:   Critical Risk Index (CRI) indicates risk by combinin...
    INSTITUTION:         IHE, Deltares
    PROVIDERS:           {"name": "clenclen", "url": "www.ivm.nl", "roles": "...
    ...                  ...
    DOI:                 https://doi.org/10.5281/zenodo.4957646
    LONG_NAME:           CRITICAL_INFRASTRUCTURE_SPATIAL_INDEX
    UNITS:               
    COMMENT:             The CISI is expressed in a dimensionless value rangi...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [15]:
# add or change certain variable / coordinate attributes
### dataset attributes is a dictionary of dictionaries
dataset_attributes = {
   # "Horizon": {"long_name": "horizon", "units": "yr"}, # set to 1 if no unit
    "lon": {"standard_name": "longitude", "long_name": "longitude", "units": "degrees_east"},
    "lat": {"standard_name": "latitude", "long_name": "latitude", "units": "degrees_north"},
    "EWL_100": { "long_name": "Extreme sea level", "units": "m"},

}  # specify custom (CF convention) attributes

 # add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        ds[k].attrs = dataset_attributes[k]
    except:
        continue
ds.drop

<bound method Dataset.drop of <xarray.Dataset>
Dimensions:   (nstations: 728088)
Coordinates:
    lon       (nstations) float64 ...
    lat       (nstations) float64 ...
    stations  (nstations) <U52 ...
Dimensions without coordinates: nstations
Data variables:
    EWL_100   (nstations) float64 ...
Attributes: (12/21)
    TITLE:               Extreme water level_ 100 RP
    TITLE_ABBREVIATION:  EWL
    DESCRIPTION:         Critical Risk Index (CRI) indicates risk by combinin...
    SHORT_DESCRIPTION:   Critical Risk Index (CRI) indicates risk by combinin...
    INSTITUTION:         IHE, Deltares
    PROVIDERS:           {"name": "clenclen", "url": "www.ivm.nl", "roles": "...
    ...                  ...
    DOI:                 https://doi.org/10.5281/zenodo.4957646
    LONG_NAME:           CRITICAL_INFRASTRUCTURE_SPATIAL_INDEX
    UNITS:               
    COMMENT:             The CISI is expressed in a dimensionless value rangi...
    CRS:                 EPSG:4326
    Conventions: 

<IPython.core.display.Javascript object>

In [16]:
# Change from string to U

stations_new=[]
for i in ds['stations'].values:
        i= i.encode('utf-8')
        stations_new.append(i)
ds.drop(["stations"])
ds = ds.assign(stations=(('nstations'),stations_new))


dataset_attributes = {
        "stations": {"long_name": "stations", "units": "1"}, # set to 1 if no unit
} 
ds

C:\Users\rowe\AppData\Local\Temp\ipykernel_9768\3047785697.py:7: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds.drop(["stations"])


<xarray.Dataset>
Dimensions:   (nstations: 728088)
Coordinates:
    lon       (nstations) float64 ...
    lat       (nstations) float64 ...
    stations  (nstations) |S52 b'Seychelles' b'Seychelles' ... b'T\xc3\xbcrkiye'
Dimensions without coordinates: nstations
Data variables:
    EWL_100   (nstations) float64 ...
Attributes: (12/21)
    TITLE:               Extreme water level_ 100 RP
    TITLE_ABBREVIATION:  EWL
    DESCRIPTION:         Critical Risk Index (CRI) indicates risk by combinin...
    SHORT_DESCRIPTION:   Critical Risk Index (CRI) indicates risk by combinin...
    INSTITUTION:         IHE, Deltares
    PROVIDERS:           {"name": "clenclen", "url": "www.ivm.nl", "roles": "...
    ...                  ...
    DOI:                 https://doi.org/10.5281/zenodo.4957646
    LONG_NAME:           CRITICAL_INFRASTRUCTURE_SPATIAL_INDEX
    UNITS:               
    COMMENT:             The CISI is expressed in a dimensionless value rangi...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [17]:
dataset_attributes = {
        "stations": {"long_name": "stations", "units": "1"}, # set to 1 if no unit
} 
# add / overwrite attributes
for k, v in dataset_attributes.items():
        try:
                ds[k].attrs = dataset_attributes[k]
        except:
                continue

<IPython.core.display.Javascript object>

In [18]:
# Write the xarray dataset to a netCDF file
#Compliant netcdf
dataset_dir_path_CF=str(dataset_dir_path).replace("original","final")


ds.to_netcdf(path=dataset_dir_path_CF)

<IPython.core.display.Javascript object>

### Check CF compliancy modified NetCDF files

In [19]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(dataset_dir_path_CF)

# check original dataset
ds

<xarray.Dataset>
Dimensions:   (nstations: 728088)
Coordinates:
    lon       (nstations) float64 ...
    lat       (nstations) float64 ...
    stations  (nstations) |S52 ...
Dimensions without coordinates: nstations
Data variables:
    EWL_100   (nstations) float64 ...
Attributes: (12/21)
    TITLE:               Extreme water level_ 100 RP
    TITLE_ABBREVIATION:  EWL
    DESCRIPTION:         Critical Risk Index (CRI) indicates risk by combinin...
    SHORT_DESCRIPTION:   Critical Risk Index (CRI) indicates risk by combinin...
    INSTITUTION:         IHE, Deltares
    PROVIDERS:           {"name": "clenclen", "url": "www.ivm.nl", "roles": "...
    ...                  ...
    DOI:                 https://doi.org/10.5281/zenodo.4957646
    LONG_NAME:           CRITICAL_INFRASTRUCTURE_SPATIAL_INDEX
    UNITS:               
    COMMENT:             The CISI is expressed in a dimensionless value rangi...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [20]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path_CF, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [21]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path_CF, working_dir=CF_dir)

{'C:\\Users\\rowe\\Documents\\GitHub\\All_Datasets\\NetCDFs\\Hazards\\EWL_100\\EWL_100_final.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [22]:
# export to zarr in write mode (to overwrite if exists)
ds.to_zarr(str(dataset_dir_path).replace("original","final").replace(".nc", ".zarr"), mode="w")

<IPython.core.display.Javascript object>